In [1]:
sc = spark.sparkContext

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1597603642565_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
nums = range(0, 10)
print(nums)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

range(0, 10)

In [3]:
rdd = sc.parallelize(nums)
    
print(sc.defaultParallelism)
print(rdd.getNumPartitions())
print(rdd.partitioner) # Right now there is no partitioner
print(rdd.glom().collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2
2
None
[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]

In [4]:
rdd = sc.parallelize(nums,15)
    
print(sc.defaultParallelism)
print(rdd.getNumPartitions())
print(rdd.partitioner) # Right now there is no partitioner
print(rdd.glom().collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2
15
None
[[], [0], [1], [], [2], [3], [], [4], [5], [], [6], [7], [], [8], [9]]

In [8]:
## Custom partitons with partitionBy()

# RDD must consist of tuple objects to use partitionBy() method

rdd = sc.parallelize(nums) \
        .map(lambda el: (el, el)) \
        .partitionBy(2).persist()
    
print(rdd.getNumPartitions())
print(rdd.partitioner)
print(rdd.glom().collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2
[[(0, 0), (2, 2), (4, 4), (6, 6), (8, 8)], [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)]]

In [9]:
### Other  way of partitioning

transactions = [
    {'name': 'Vishwa', 'amount': 100, 'state': 'Bihar'},
    {'name': 'Mohan', 'amount': 15, 'state': 'Bihar'},
    {'name': 'Shivani', 'amount': 51, 'state': 'Karnataka'},
    {'name': 'Abhishek', 'amount': 200, 'state': 'TamilNadu'},
    {'name': 'Rohan', 'amount': 75, 'state': 'Karnataka'},
]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
def state_partitioner(state):
    return hash(state)

num_partitions = 5
print(state_partitioner("Poland") % num_partitions)
print(state_partitioner("Germany") % num_partitions)
print(state_partitioner("United Kingdom") % num_partitions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0
0
1

In [11]:
rdd = sc.parallelize(transactions) \
        .map(lambda el: (el['state'], el)) \
        .partitionBy(4, state_partitioner)
    
print(rdd.getNumPartitions())
print(rdd.partitioner)
print(rdd.glom().collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4
[[('Bihar', {'name': 'Vishwa', 'amount': 100, 'state': 'Bihar'}), ('Bihar', {'name': 'Mohan', 'amount': 15, 'state': 'Bihar'}), ('Karnataka', {'name': 'Shivani', 'amount': 51, 'state': 'Karnataka'}), ('Karnataka', {'name': 'Rohan', 'amount': 75, 'state': 'Karnataka'})], [('TamilNadu', {'name': 'Abhishek', 'amount': 200, 'state': 'TamilNadu'})], [], []]

In [12]:
### partitioning in the case of the dataframe
from pyspark.sql import Row
rdd = sc \
        .parallelize(transactions) \
        .map(lambda x: Row(**x))
df = spark.createDataFrame(rdd)
df.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+---------+
|amount|    name|    state|
+------+--------+---------+
|   100|  Vishwa|    Bihar|
|    15|   Mohan|    Bihar|
|    51| Shivani|Karnataka|
|   200|Abhishek|TamilNadu|
|    75|   Rohan|Karnataka|
+------+--------+---------+

In [13]:
df2 = df.repartition("state")
print("\nAfter 'repartition()'")
print(df2.rdd.getNumPartitions())
print(df2.rdd.partitioner)
print(df2.rdd.glom().collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


After 'repartition()'
200
None
[[], [], [], [], [], [Row(amount=51, name='Shivani', state='Karnataka'), Row(amount=75, name='Rohan', state='Karnataka')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [Row(amount=200, name='Abhishek', state='TamilNadu')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [Row(amount=100, name='Vishwa', state='Bihar'), Row(amount=15, name='Mohan', state='Bihar')], 

In [14]:
#repartition  - when increasing the number of parttitions and coalesce when decreasing the number of partitions

df2= df2.coalesce(10)
print("\nAfter 'coalescing()'")
print(df2.rdd.getNumPartitions())
print(df2.rdd.partitioner)
print(df2.rdd.glom().collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


After 'coalescing()'
10
None
[[Row(amount=51, name='Shivani', state='Karnataka'), Row(amount=75, name='Rohan', state='Karnataka')], [Row(amount=100, name='Vishwa', state='Bihar'), Row(amount=15, name='Mohan', state='Bihar')], [], [Row(amount=200, name='Abhishek', state='TamilNadu')], [], [], [], [], [], []]